In [1]:
subscription_id = 'a89228ac-8bf4-4646-9d2c-442b1cb5d622'
resource_group = 'lauri-ml'
aml_workspace = 'lauri-ml'
cluster_name = 'bert-gpu'
# cluster_vmsize = 'Standard_NC12s_v3'
cluster_vmsize = 'Standard_NC6'

In [2]:
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget

## Connect To Workspace

In [ ]:
# workspace = Workspace(
#     subscription_id=subscription_id, resource_group=resource_group, workspace_name=aml_workspace
# )
# workspace.write_config()

In [4]:
# from azureml.core.authentication import InteractiveLoginAuthentication
# interactive_auth = InteractiveLoginAuthentication()
# workspace = Workspace.from_config(auth=interactive_auth)

workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep = '\n')

Performing interactive authentication. Please follow the instructions on the terminal.
WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
WARNING - You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: lauri-ml
Azure region: westeurope
Subscription id: a89228ac-8bf4-4646-9d2c-442b1cb5d622
Resource group: lauri-ml


## Create Compute Target

A [compute target](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.computetarget?view=azure-ml-py) is a designated compute resource/environment where you run your training script or host your service deployment. This location may be your local machine or a cloud-based compute resource. Compute targets can be reused across the workspace for different runs and experiments. 

For this tutorial, we will create an auto-scaling [Azure Machine Learning Compute](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute?view=azure-ml-py) cluster, which is a managed-compute infrastructure that allows the user to easily create a single or multi-node compute. To create the cluster, we need to specify the following parameters:

- `vm_size`: The is the type of GPUs that we want to use in our cluster. For this tutorial, we will use **Standard_NC12s_v3 (NVIDIA V100) GPU Machines** .
- `idle_seconds_before_scaledown`: This is the number of seconds before a node will scale down in our auto-scaling cluster. We will set this to **6000** seconds. 
- `min_nodes`: This is the minimum numbers of nodes that the cluster will have. To avoid paying for compute while they are not being used, we will set this to **0** nodes.
- `max_modes`: This is the maximum number of nodes that the cluster will scale up to. Will will set this to **2** nodes.

**When jobs are submitted to the cluster it takes approximately 5 minutes to allocate new nodes** 

#### Use AML SDK to create the compute target

In [7]:
compute_config = AmlCompute.provisioning_configuration(
    vm_size=cluster_vmsize,
    idle_seconds_before_scaledown=6000,
    min_nodes=0,
    max_nodes=2
)

compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


To ensure our compute target was created successfully, we can check it's status.

In [13]:
compute_target.get_status().serialize()

{'allocationState': 'Steady',
 'allocationStateTransitionTime': '2020-02-10T22:15:59.775000+00:00',
 'creationTime': '2020-02-03T17:36:49.632013+00:00',
 'currentNodeCount': 0,
 'errors': None,
 'modifiedTime': '2020-02-03T17:37:05.352620+00:00',
 'nodeStateCounts': {'idleNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0,
  'preparingNodeCount': 0,
  'runningNodeCount': 0,
  'unusableNodeCount': 0},
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'maxNodeCount': 2,
  'minNodeCount': 0,
  'nodeIdleTimeBeforeScaleDown': 'PT6000S'},
 'targetNodeCount': 0,
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_NC6'}